# Convert Word (Transcript) to DataFrame

[Click here](https://colab.research.google.com/github/senthilchandrasegaran/IDEM105-EDI/blob/main/word-transcript-to-excel.ipynb) to open this notebook in Colab.

Transcription using MS Word creates a format where timestamp and speaker are on one line, and the utterance is on the next. This notebook converts this format (from a Word docx file) to a CSV or Excel file by first converting it into a Pandas DataFrame first.

In [1]:
# !pip install docx2txt
import docx2txt
import pandas as pd
from IPython.display import display, Markdown

### Get relevant text from the Word Doc
The Word document often contains some preamble, typically followed by the word `Transcript` after which the transcript begins. Test this assumption by printing the contents of the variable `text` below. If this is correct, then only extract the text _after_ the word 'Transcript'.

In [107]:
transcript_path = "data/transcripts_2025-26/Design session_Yen-Hsin-Yumin.docx"
transcript = docx2txt.process(transcript_path)
# print the first 500 characters, just to check.
transcript[:500]

"Yumin_Kye 0:04\n\nI started recording. You started transcription.\n\nYen-Hsin_Lin 0:08\n\nYeah, you can see the live transcription on your right hand side. Uh, if you click on transcript or something.\n\nYumin_Kye 0:24\n\nLet me see. Yeah, I see. Yeah.\n\nYen-Hsin_Lin 0:35\n\nDid you see that? Yeah, but I'm gonna close it for now because I'm gonna be distracted by that. OK, let's do it.\n\nYumin_Kye 0:49\n\nOK, uh. Right. Uh, just give me a sec.\n\nYen-Hsin_Lin 0:56\n\nYeah.  Do you want to share the screen or?\n\nYumi"

### Clean up the text
A number of carriage returns will be in the text file. This is useful to split up the file into lines, so the code below does the following:
1. Splits the text into lines where a line is either a `timestamp, speaker` combo an `utterance`, or an empty line (in case of multiple successive carriage returns;
2. Removes all empty lines;
3. Replaces all non-breaking spaces (unicode: `\xa0`)
4. Splits the `'timestamp, speaker'` string combination into a list with `['timestamp', 'speaker']` in it.

In [108]:
transcript_list = transcript.split("\n")
transcript_list = [s.strip() for s in transcript_list if len(s.strip()) > 0]
transcript_list = [s.replace('\xa0',  ' ') for s in transcript_list]
transcript_list[0:10] # print the entire transcript (i.e., remove the range of [0:10]) if you want to check your entire transcript.

['Yumin_Kye 0:04',
 'I started recording. You started transcription.',
 'Yen-Hsin_Lin 0:08',
 'Yeah, you can see the live transcription on your right hand side. Uh, if you click on transcript or something.',
 'Yumin_Kye 0:24',
 'Let me see. Yeah, I see. Yeah.',
 'Yen-Hsin_Lin 0:35',
 "Did you see that? Yeah, but I'm gonna close it for now because I'm gonna be distracted by that. OK, let's do it.",
 'Yumin_Kye 0:49',
 'OK, uh. Right. Uh, just give me a sec.']

In [109]:
print("Remove last item if it is incomplete.")
print('--------------------------- LAST 4 LINES ---------------------------')
for t in transcript_list[-4:] :
    print(t)
print('-------------------------------- END -------------------------------')

# Uncomment and re-run this cell if the last line is found to be incomplete
# if type(transcript_list[-1]) == list :
    # transcript_list.pop() 

Remove last item if it is incomplete.
--------------------------- LAST 4 LINES ---------------------------
Yumin_Kye 1:02:32
Yeah, I think we can wrap up maybe.
Yen-Hsin_Lin 1:02:32
Yeah.
-------------------------------- END -------------------------------


### Convert into DataFrame
Convert the list with successive items being `['timestamp', 'speaker]` and `'utterance'` items into a DataFrame

In [110]:
transcript_triad = []
for ind, line in enumerate(transcript_list) :
    if ind % 2 == 0 and ind+1 < len(transcript_list):
        # these are speaker/timestamp lines
        if line[0].isdigit() :
            # the start is a timestamp
            timestamp = line.split(' ')[0]
            speaker = ' '.join(line.split(' ')[1:])
            utterance = transcript_list[ind+1]
        else :
        # The line contains speaker but not timestamps 
            timestamp = ''
            speaker = line
            if len(line.split(' ')) > 2 :
                print("*****************************************")
                print("Problem found!")
                print(transcript_list[ind-3:ind+1])
                print(ind)
                print("*****************************************")
                break
            utterance = transcript_list[ind+1]
        turn_data = { 'timestamp' : timestamp,
                      'speaker'   : speaker,
                      'utterance' : utterance
                    }
        transcript_triad.append(turn_data)
            

In [114]:
# print(transcript_triad)
df = pd.DataFrame(transcript_triad)
df.sample(5)

,timestamp,speaker,utterance
24,,Yumin_Kye 4:09,And I will just quickly explain about the back...
392,,Yumin_Kye 1:00:43,Yeah.
325,,Yen-Hsin_Lin 48:31,"…it won't be too noisy? I don't know, just ju..."
13,,Yen-Hsin_Lin 1:46,Yeah.
152,,Yumin_Kye 23:51,This sign up process field because my initial ...


### Save DataFrame
Because this involves text which can have punctuations including commas and semicolons, a CSV file is not recommended. Instead, save it as an excel file.

In [112]:
excel_name = transcript_path.split(".docx")[0] + ".xlsx"
df.to_excel(excel_name, index=False)
print("File saved to:", excel_name)

File saved to: data/transcripts_2025-26/Design session_Yen-Hsin-Yumin.xlsx
